Basic genetic regression to estimate a function by a polynomial

Below is the code for standard linear regression. We use the same dataset to check genetic regression

In [1]:
import matplotlib.pyplot as plt 
import numpy as np 
from sklearn import datasets, linear_model, metrics 

# load the boston dataset 
boston = datasets.load_boston(return_X_y=False) 

# defining feature matrix(X) and response vector(y) 
X = boston.data 
y = boston.target 

# splitting X and y into training and testing sets 
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, 
													random_state=1) 

# create linear regression object 
reg = linear_model.LinearRegression() 

# train the model using the training sets 
reg.fit(X_train, y_train) 

# regression coefficients 
print('Coefficients: \n', reg.coef_) 

# variance score: 1 means perfect prediction 
print('Variance score: {}'.format(reg.score(X_test, y_test))) 

# plot for residual error 

## setting plot style 
plt.style.use('fivethirtyeight') 

## plotting residual errors in training data 
plt.scatter(reg.predict(X_train), reg.predict(X_train) - y_train, 
			color = "green", s = 10, label = 'Train data') 

## plotting residual errors in test data 
plt.scatter(reg.predict(X_test), reg.predict(X_test) - y_test, 
			color = "blue", s = 10, label = 'Test data') 

## plotting line for zero residual error 
plt.hlines(y = 0, xmin = 0, xmax = 50, linewidth = 2) 

## plotting legend 
plt.legend(loc = 'upper right') 

## plot title 
plt.title("Residual errors") 

## function to show plot 
plt.show() 


Coefficients: 
 [-8.95714048e-02  6.73132853e-02  5.04649248e-02  2.18579583e+00
 -1.72053975e+01  3.63606995e+00  2.05579939e-03 -1.36602886e+00
  2.89576718e-01 -1.22700072e-02 -8.34881849e-01  9.40360790e-03
 -5.04008320e-01]
Variance score: 0.720905667266178


<Figure size 640x480 with 1 Axes>

We will use the same dataset to predict the output for genetic regression.



In [20]:
#testing matrix - vector
mat = np.array([[1,2,3],[4,5,6]]);
colvec = np.array([[1],[2]]);
rowvec = np.array([[1,2,3]]);
array = np.array([1,2,3]);
#print(mat.shape,colvec.shape,rowvec.shape,array.shape);
#mat+array and mat+rowvec adds the row to each col of matrix
#mat + colvec adds the column to each column of matrix
print(mat+rowvec,"\n\n",mat+array,"\n\n",mat+colvec);

# converting array to column vector
a = np.array([1, 2, 3, 4])
a = a.reshape((-1, 1)) # <--- THIS IS IT

#print (a)

[[2 4 6]
 [5 7 9]] 

 [[2 4 6]
 [5 7 9]] 

 [[2 3 4]
 [6 7 8]]


In [21]:
import matplotlib.pyplot as plt 
import numpy as np 
from sklearn import datasets, linear_model, metrics 

# load the boston dataset 
boston = datasets.load_boston(return_X_y=False) 

# defining feature matrix(X) and response vector(y) 
X = boston.data 
y = boston.target 

# splitting X and y into training and testing sets 
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, 
													random_state=1) 



In [22]:
trainX = np.array(X_train)
trainY = np.array(y_train)
testX = np.array(X_test)
testY = np.array(y_test)

In [33]:
#initializing some constants
POOL_SIZE = 1000;
#INITIAL_CANDIDATES = 30
THETA_RANGE_LOW = -10;
THETA_RANGE_HIGH = 10;
ELIMINATION_RATE = 0.2; #after each round lowest scoring candidates are eliminated with this rate
ROUNDS = 1000;
ALL_RANDOM_CHILD = 0.1; # ratio of the children who will be all random

In [39]:
# initialize candidates
# col no of candidates are initialized, each with a random value between low and high
# last column for saving fitness
def init_candidates(row,col,low,high):
    return (high-low)*(np.random.random(size=(row,col+1)))+low;

In [40]:
candidates = init_candidates(POOL_SIZE,trainX.shape[1],THETA_RANGE_LOW,THETA_RANGE_HIGH);
print(trainX.shape);
print(candidates.shape);
print(trainY.shape);

(303, 13)
(1000, 14)
(303,)


In [41]:
"""
evaluates the avg error of a candidate in approximating y
returns the average error for each candidate
considers the mean average error
"""
def fitness(candidates,trainingX,trainingY):
    #print(candidate);
    predictionMatrix = np.dot(trainingX,np.transpose(candidates[:,:-1])); # each column corresponds to the results for one element against all the examples
    #print(candidates.shape);
    #print(trainingX.shape);
    #print(trainingY.shape);
    #print(predictionMatrix.shape);
    
    diff = predictionMatrix - trainingY.reshape((-1,1));#now each column has the difference from prediction reshape is used to turn array into column vector
    diff=np.abs(diff); 
    sumOfErr = diff.sum(axis=0); #column wise sum
    #print(sumOfErr.shape);
    #print(trainingX.shape,trainingX.shape[0],trainingX.shape[1]);
    avgErr = -sumOfErr/trainingX.shape[0];
    candidates[:,-1] = avgErr;
    #print(avgErr);
    return np.array(avgErr);
    
"""
having passed parents matrix, makes count number of children through crossover
and random mutation
"""
import random;

"""
Having passed a vector mother and father
returns the child after breeding
"""

def breed(mother,father):
    #print(mother.shape,father.shape);
    len = mother.shape[0];
    crossOverPoint = random.randint(0,len-1);
    #print(crossOverPoint," coPoint");
    child = np.zeros(len);
    #print(child.shape)
    i=0;
    while(i<len):
        if(i<crossOverPoint):
            child[i] = mother[i];
        else:
            child[i] = father[i];
        i=i+1;
            
    rand = random.randint(1,10);
    if(rand<2):
        #mutation
        # pick mutation point
        mp = random.randint(0,len-1);
        mv = random.randint(-10,10); #this value will be added
        child[mp] = child[mp]+mv;
    return child;


"""
having passed a matrix parents, one candidate each row
generates count no of offsprings
returns as a matrix
"""
def makeChildren(parents,count):
    tempArr = np.zeros((count,parents.shape[1]));
    #print(tempArr.shape);
    k =0;
    while(count>0):
        i = random.randint(0,parents.shape[0]-1);
        j = random.randint(0,parents.shape[0]-1);
        if(i!=j):
            child = breed(parents[i],parents[j]);
            #print(child.shape);
            tempArr[k,:] = child;
            k=k+1;
            count = count-1;
    return tempArr


In [42]:

def eliminate_populate(candidates,fitness):
    #merge fitness with candidates
    #print(fitnessArr.shape);
    #fitnessArr =fitness.reshape((-1,1));
    
    #print(fitnessArr.shape);
    #print(candidates.shape);
    #candidates = np.append(candidates,fitnessArr,axis=0);
    #concat = np.hstack((candidates,fitnessArr)); # concats the column
    
    #candidates = concat;
    candidates = candidates[candidates[:,candidates.shape[1]-1].argsort()]; # sort by the fitness(last column)
    #print(candidates.shape);
    
    print("Fitness: ",candidates[-1,-1]);
    # now the array is sorted based on lowesrt fitness first row
    #print(candidates);
    deaths = int(POOL_SIZE*ELIMINATION_RATE);
    allRandomBirthNo = int(deaths*ALL_RANDOM_CHILD);
    normalBirthNo = int(deaths - allRandomBirthNo);
    
    #separate survived population
    survived = candidates[deaths:,:-1];
    newBees = makeChildren(survived,deaths);
    
    # append one extra column to contain fitness value
    #print(newBees.shape);
    a = np.zeros(newBees.shape[0]);
    #print(a.shape);
    a = a.reshape((-1,1)); 
    #print(a.shape);
    newBees = np.hstack((newBees,a));
    
    #print(candidates.shape);
    candidates = np.vstack((candidates[deaths:,:],newBees));
    #print(candidates.shape);
    
    #print(newBees.shape);
    #print(newBees);
    #print(survived);
    return candidates;
    
# returns an array of candidates 
#def eliminate(candidates,fitness)
def geneticAlgo(candidates,trainX,trainY):
    count = 0;
    #calculate fitness for every 
    while(count<10000):
        fitnessArr = fitness(candidates,trainX,trainY);
        candidates = eliminate_populate(candidates,fitnessArr);
        count = count+1;
    
    return candidates[1,:-1]
    #eliminated = eliminate(candidates,fitness); 


In [43]:
messiah = geneticAlgo(candidates,trainX,trainY)

Fitness:  -1834.1305187087112
Fitness:  -1834.1305187087112
Fitness:  -1834.1305187087112
Fitness:  -1834.1305187087112
Fitness:  -1814.0021839732233
Fitness:  -1439.265704211348
Fitness:  -1439.265704211348
Fitness:  -1439.265704211348
Fitness:  -1157.7378967085826
Fitness:  -1157.7378967085826
Fitness:  -1157.7378967085826
Fitness:  -1157.7378967085826
Fitness:  -1157.7378967085826
Fitness:  -1157.7378967085826
Fitness:  -708.3683067184561
Fitness:  -708.3683067184561
Fitness:  -708.3683067184561
Fitness:  -708.3683067184561
Fitness:  -708.3683067184561
Fitness:  -595.7763089721399
Fitness:  -595.7763089721399
Fitness:  -595.7763089721399
Fitness:  -595.7763089721399
Fitness:  -545.3744362059408
Fitness:  -545.3744362059408
Fitness:  -545.3744362059408
Fitness:  -545.3744362059408
Fitness:  -545.3744362059408
Fitness:  -545.3744362059408
Fitness:  -545.3744362059408
Fitness:  -539.809744754995
Fitness:  -539.809744754995
Fitness:  -539.809744754995
Fitness:  -501.0288633963097
Fitnes

Fitness:  -152.48721575219884
Fitness:  -152.48721575219884
Fitness:  -152.48721575219884
Fitness:  -152.07814011563772
Fitness:  -152.07814011563772
Fitness:  -152.07814011563772
Fitness:  -152.07814011563772
Fitness:  -152.07814011563772
Fitness:  -152.07814011563772
Fitness:  -152.07814011563772
Fitness:  -152.07814011563772
Fitness:  -152.07814011563772
Fitness:  -152.07814011563772
Fitness:  -152.07814011563772
Fitness:  -152.07814011563772
Fitness:  -152.07814011563772
Fitness:  -152.07814011563772
Fitness:  -152.07814011563772
Fitness:  -152.07814011563772
Fitness:  -152.07814011563772
Fitness:  -152.07814011563772
Fitness:  -152.07814011563772
Fitness:  -152.07814011563772
Fitness:  -152.07814011563772
Fitness:  -152.07814011563772
Fitness:  -152.07814011563772
Fitness:  -152.07814011563772
Fitness:  -152.07814011563772
Fitness:  -152.07814011563772
Fitness:  -152.07814011563772
Fitness:  -152.07814011563772
Fitness:  -152.07814011563772
Fitness:  -152.07814011563772
Fitness:  

Fitness:  -143.02810207295732
Fitness:  -143.02810207295732
Fitness:  -143.02810207295732
Fitness:  -143.02810207295732
Fitness:  -142.90464637075112
Fitness:  -142.83425165127923
Fitness:  -142.38059345223863
Fitness:  -142.38059345223863
Fitness:  -142.38059345223863
Fitness:  -142.38059345223863
Fitness:  -142.38059345223863
Fitness:  -142.38059345223863
Fitness:  -142.38059345223863
Fitness:  -142.38059345223863
Fitness:  -142.30769766157684
Fitness:  -142.30769766157684
Fitness:  -142.30769766157684
Fitness:  -142.30769766157684
Fitness:  -142.30769766157684
Fitness:  -142.30769766157684
Fitness:  -142.25249661291002
Fitness:  -142.25249661291002
Fitness:  -142.25249661291002
Fitness:  -142.25249661291002
Fitness:  -142.25249661291002
Fitness:  -142.25249661291002
Fitness:  -142.25249661291002
Fitness:  -142.25249661291002
Fitness:  -142.25249661291002
Fitness:  -142.24176178749008
Fitness:  -142.24176178749008
Fitness:  -142.24176178749008
Fitness:  -142.24176178749008
Fitness:  

Fitness:  -138.20165171179937
Fitness:  -138.20165171179937
Fitness:  -138.20165171179937
Fitness:  -138.20165171179937
Fitness:  -138.16416128275637
Fitness:  -138.16416128275637
Fitness:  -137.91360624445113
Fitness:  -137.91360624445113
Fitness:  -137.91360624445113
Fitness:  -137.91360624445113
Fitness:  -137.91360624445113
Fitness:  -137.52308860190203
Fitness:  -137.52308860190203
Fitness:  -137.52308860190203
Fitness:  -137.39871529234063
Fitness:  -137.39871529234063
Fitness:  -137.39871529234063
Fitness:  -137.39871529234063
Fitness:  -137.39871529234063
Fitness:  -137.39871529234063
Fitness:  -137.39871529234063
Fitness:  -137.39871529234063
Fitness:  -137.39871529234063
Fitness:  -137.39871529234063
Fitness:  -137.39309450778774
Fitness:  -137.39309450778774
Fitness:  -137.39309450778774
Fitness:  -137.39309450778774
Fitness:  -137.39309450778774
Fitness:  -137.39309450778774
Fitness:  -137.39309450778774
Fitness:  -137.39309450778774
Fitness:  -137.39309450778774
Fitness:  

Fitness:  -132.25454349637818
Fitness:  -132.16156182418487
Fitness:  -132.16156182418487
Fitness:  -132.16156182418487
Fitness:  -132.16156182418487
Fitness:  -132.16156182418487
Fitness:  -132.16156182418487
Fitness:  -132.16156182418487
Fitness:  -132.16156182418487
Fitness:  -132.16156182418487
Fitness:  -132.10231989284938
Fitness:  -132.10231989284938
Fitness:  -132.10231989284938
Fitness:  -132.10231989284938
Fitness:  -132.10231989284938
Fitness:  -132.10231989284938
Fitness:  -132.10231989284938
Fitness:  -132.10231989284938
Fitness:  -132.10231989284938
Fitness:  -132.10231989284938
Fitness:  -132.10231989284938
Fitness:  -132.10231989284938
Fitness:  -132.10231989284938
Fitness:  -132.03803006794487
Fitness:  -132.03803006794487
Fitness:  -132.03803006794487
Fitness:  -132.03803006794487
Fitness:  -132.03803006794487
Fitness:  -132.03803006794487
Fitness:  -132.03803006794487
Fitness:  -131.99094795957987
Fitness:  -131.99094795957987
Fitness:  -131.99094795957987
Fitness:  

...

Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  

Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  

Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  

Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  

Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  -123.07656575747001
Fitness:  

In [29]:
print(messiah)

[ 3.47879379e-01  4.75959013e-02 -1.58185276e+00  7.50354612e+00
  1.51330733e+01  5.04313661e+00 -5.80025032e-01 -6.24329974e+00
 -4.58183909e-01 -1.50843683e-03  3.38951415e+00 -7.07823035e-05
  3.28668972e-01]


In [30]:
contender = reg.coef_;
print(contender);

[-8.95714048e-02  6.73132853e-02  5.04649248e-02  2.18579583e+00
 -1.72053975e+01  3.63606995e+00  2.05579939e-03 -1.36602886e+00
  2.89576718e-01 -1.22700072e-02 -8.34881849e-01  9.40360790e-03
 -5.04008320e-01]


In [31]:
def fitness2(candidates,trainingX,trainingY):
    #print(candidate);
    predictionMatrix = np.dot(trainingX,np.transpose(candidates)); # each column corresponds to the results for one element against all the examples
    #print(candidates.shape);
    #print(trainingX.shape);
    #print(trainingY.shape);
    #print(predictionMatrix.shape);
    
    diff = predictionMatrix - trainingY.reshape((-1,1));#now each column has the difference from prediction reshape is used to turn array into column vector
    diff=np.abs(diff); 
    sumOfErr = diff.sum(axis=0); #column wise sum
    #print(sumOfErr.shape);
    #print(trainingX.shape,trainingX.shape[0],trainingX.shape[1]);
    avgErr = -sumOfErr/trainingX.shape[0];
    #candidates[:,-1] = avgErr;
    #print(avgErr);
    return np.array(avgErr);

messiahFit = fitness2(messiah,testX,testY);
print(messiahFit);

[-18.36507292  -9.24802671 -16.83675078 -16.49349676 -20.96076822
 -14.38667724 -31.36100938  -9.00226739 -13.10474197 -16.8333235
 -13.12110202 -18.21502501  -8.86327354 -10.27571554  -8.21994029
 -15.00457921  -6.62915854 -11.811737    -8.71283224 -23.30908443
  -8.92621184 -26.90309442  -8.79149368 -19.01255786 -16.62743432
  -9.84862699  -6.66263391  -8.02843849  -6.67266692 -16.04992915
 -10.4705122  -19.83519861 -22.51482255 -13.63449657 -14.10895017
 -26.94703432 -13.13101686 -12.92028849 -15.01804473 -17.21656656
 -25.42929265 -15.3277388   -6.6427656  -15.57144558 -15.04933966
  -9.72829937 -19.65789195 -11.55157336 -31.255887   -11.48951609
  -6.69652361  -9.70755106  -8.66822126 -30.60652389 -25.3329217
 -10.72127161  -8.24575684 -16.84076028  -7.64920447 -11.75563098
 -13.11093303 -14.67187791 -14.29533314  -9.46874664 -37.28211496
 -24.54700789  -9.82016412  -6.94670522 -12.0378419  -32.31746505
 -20.4171175  -11.107679   -17.51917977  -7.40708009 -14.05224489
  -6.7803056

In [32]:
con = fitness2(contender,testX,testY);
print(con);

[-24.37542743 -28.6363706  -38.92730208 -35.72497201 -38.64514802
 -36.5336543  -23.94832466 -38.71728738 -32.79224698 -30.50832168
 -30.05633658 -27.8381643  -36.47817574 -30.22930369 -33.94303324
 -36.89185096 -39.01606236 -18.34302945 -26.73372505 -48.2271025
 -36.53033173 -41.32294748 -31.80648987 -32.13134453 -25.6196642
 -46.63109484 -42.97920994 -40.35280094 -20.84109879 -42.44240178
 -35.19296642 -42.77135597 -13.2804692  -38.66153225 -35.87528913
 -36.58686092 -39.18028561 -29.54064029 -48.16704811 -37.18009829
 -33.37619402 -35.46742725 -27.94096952 -40.66119918 -37.60777589
 -42.23119024 -17.85052499 -38.79569296 -32.04314944 -37.34384393
 -31.60829029 -32.60120554 -32.10060118 -31.05255013 -52.3379922
 -32.55361897 -45.68143259 -30.54446885 -39.84251717 -20.86180473
 -37.27556136 -29.71283839 -40.41856087 -38.79313126 -45.64865339
 -24.97022287 -19.99381888 -35.49311959 -32.8197012  -32.53724801
 -33.48975212 -49.96676904 -41.26216818 -36.04319301 -36.30120185
 -35.86041147